# Part A: Produce a monthly image composite

*   Part A: produce a median composite for a specific date range
*   Part B: extract a time series and plot
*   Part C: produce a monthly median, extract and plot

In [ ]:
!pip install geemap

import ee
import numpy as np
import geemap.eefolium as geemap

ee.Authenticate()
ee.Initialize()

We are going to create a median composite of Sentinel-2 imagery.  
We will use the following cloud mask:

In [2]:
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000); # Divide by 10000 takes scale factor into account!

My AOI is the Debeers Victor Mine in Northern Ontario.  I'm using a coordinate to define that but you could use a feature collection instead. 

In [3]:
victor = ee.Geometry.Point([-83.90,52.81])

S2_Victor2019 = ee.ImageCollection("COPERNICUS/S2_SR")\
                    .filterBounds(victor)\
                    .filterDate('2019-06-01', '2019-08-01')\
                    .map(maskS2clouds)


We are using a "reducer" to calculate the median pixel-wise from all images in our image collection.

In [4]:
VictorMedian = S2_Victor2019.reduce(ee.Reducer.median())

Set up the plot.

In [ ]:
rgbVis = {
  'min': 0.0,
  'max': 0.1,
  'bands': ['B4_median', 'B3_median', 'B2_median'],
};

Map = geemap.Map()
Map.centerObject(victor, 10)
Map.addLayer(VictorMedian, rgbVis, 'RGB Color')
Map.add_layer_control()
Map

# Part B: Create a function to extract the mean value at a point for all images in the image collection, and plot it
Adapted from 
https://worldbank.github.io/OpenNightLights/tutorials/mod4_1_time_series_charts.html

In [6]:
# we are going to use some additional packages
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
    

In [7]:
def poi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=poi, scale=30, maxPixels = 100000).get(band)  # This calculates the mean of the pixels at the location of the POI
    return img.set('date', img.date().format()).set('mean',mean)

In [8]:
poi = victor
# poi = ee.FeatureCollection('users/koreenmillard/Alfred_classification_poly')  # I've made this publically available, but you can change it to your own file if you want
# You can try this with a polygon as well. Just note that it will be slower.
# maxPixels has to be set so if you use a very large polygon it will sample the pixels within the polygon up to the number 

MODIS = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate('2014-01-01','2015-5-31')
band = 'EVI'
poi_reduced_imgs = MODIS.map(poi_mean)

So what did we just do here? 

Let's explore...

In [15]:
# Print out all the band ids for an image json
def printImageBands(image):
  print('Bands of example image:')
  for band in range(len(image['bands'])):
    print(image['bands'][band]['id'])
  print('\n')

# Print the size of a json image collection
def printCollectionSize(collection):
  print('Size of image collection:')
  print(len(collection['features']))
  print('\n')

# Get an image json object from a json image collection based on an index value
def getImageFromCollection(collection, index=0):
  return collection['features'][index]

# Print all image ids from a json image collection
def printImageNames(collection):
  print('All images in image collection:')
  for img in range(len(collection['features'])):
    print(str(img) + ': ' + collection['features'][img]['id'])
  print('\n')

In [ ]:
# Print section
print("let's look at the original MODIS image first")

# First, we need to use the getInfo function to get all the information about our image Collection into json format
stack_json = MODIS.getInfo()

# Then, we can run our neat print functions on new variable.
# printCollectionSize tells us how many images are in our image collection
# printImageNames shows all the image ids from every image in the collection
printCollectionSize(stack_json)
printImageNames(stack_json)

# If we want to look at a specific image in the collection, we need to first grab an image from our json variable and put it in a new variable
img_json = getImageFromCollection(stack_json)

# Then we can look at what bands are in this image
printImageBands(img_json)

# Get a list of all metadata properties
properties = poi_reduced_imgs.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties

print('         ')
print("now let's look at the poi_reduced_images results")
# First, we need to use the getInfo function to get all the information about our image Collection into json format
stack_json = poi_reduced_imgs.getInfo()

# Then, we can run our neat print functions on new variable.
# printCollectionSize tells us how many images are in our image collection
# printImageNames shows all the image ids from every image in the collection
printCollectionSize(stack_json)
printImageNames(stack_json)

# If we want to look at a specific image in the collection, we need to first grab an image from our json variable and put it in a new variable
img_json = getImageFromCollection(stack_json)

# Then we can look at what bands are in this image
printImageBands(img_json)

# Get a list of all metadata properties
properties = poi_reduced_imgs.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties



So the new image collection remains unchanged it seems...?

Let's look closer at a specific image in the image collection

In [16]:
# Get an image from an image collection based on the index of the image
# Note: This is working with images and image collections, not json
# index = 0 is set so that it defaults to the first image if you don't set an imageID when you call this function
def getImageByIndex(collection, index=0):  

  # Convert the image collection to a list
  listOfImages = collection.toList(collection.size())

  # Return an image based on its index
  # This must be cast to the ee.Image type
  return ee.Image(listOfImages.get(index))

# Let's get a single image (the second one) from our image collection
SecondImage = getImageByIndex(poi_reduced_imgs, 1)

In [ ]:
# Get a list of all metadata properties.
properties = SecondImage.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties

So there are your date and mean properties - stored with each image in your image collection.

## Now, let's plot our extracted data

The below text explains the upcoming code.  

`nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)`

Here's what's happening with nested_list. You've got the image collection (poi_reduced_imgs) and for each image, we have stored mean of the EVI pixel and the date corresponding to the image (i.e. Both the mean and the date are stored as properties of the images).

**Let's look at what is happening here**

`ee.Reducer.toList(2)`

>This is a reducer that returns tuples (pairs) of size 2. Basically this is telling EE that each image will have both date and mean values.

`['date','mean']`

>This is a list of the chosen properties of the image that are grouped together in the reducer. These can be changed to any other property that the images hold (but all images must have these properties).  

`reduceColumns()`

>This is the reducer mapping that is done over every image in the collection.  The difference between it and the standard reducer is that this one is deliberatly getting the results into something that can be formed into a table.

`poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean'])`

>This gets you the list of the properties you want in the format that will work for a pandas table. However this is still an EE.Object, which cannot be used with a pandas dataframe.

`poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values()`

>Using .values() turns the EE object into something stored in a list.It is now not an EE.Object so pandas can use it, but all the tuples are stored in a list inside another list. This basically is what we need except it is stuffed inside an extra list. 

`poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)`

>The .get(0) gets everything stored in the first spot in the outer list. Since every tuple that we wanted is stored in a list that itself is stored at the first index in the outer list, using .get(0) gets everything we need.It removes the extra list and gives us a list of all the tuples, which is the right format to be used with pandas dataframe.

In [ ]:
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)

df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])

df

In [ ]:
# gca stands for 'get current axis'
ax = plt.gca()

df.plot(kind='line',x='date',y='mean', color='red', ax=ax, figsize=(25,7))
plt.title('NIR for Victor Mine (Jan 2014 to May 2015)',fontsize=20)
plt.xlabel('date', fontsize=20)
plt.ylabel('monthly mean EVI', fontsize=20)
plt.show()

---

# Part C: create a monthly median composite





In [20]:
# Get the image collection
collection =  ee.ImageCollection("MODIS/006/MOD10A1")  # We will use the daily snow cover dataset

# Create Monthly Composite 
months = ee.List.sequence(1, 12) 

# Function for getting the median value of the month and setting the month value as a property
# This is a new version of the function above - note the addition of the system:time_start
# This is required in order to extract the date from the images, but is wiped when we calculate the median
# So instead, here we use the month's "start" date, which is set as a variable in the function
def getMonthMedian(m):
  start = ee.Date.fromYMD(2018, m, 1)
  end = start.advance(1, 'month')
  image = collection.filterDate(start, end).median()
  return image.set('month', m).set('system:time_start', ee.Date(start).millis())  

new = ee.ImageCollection(months.map(getMonthMedian))

In [ ]:
# let's check how many images are in the collection, and what bands it contains using the functions we defined earlier
# Print section
stack_json = new.getInfo()
printCollectionSize(stack_json)
img_json = getImageFromCollection(stack_json)
printImageBands(img_json)

print(img_json['properties'])

In [22]:
band = 'NDSI_Snow_Cover'
poi_reduced_imgs = new.map(poi_mean)

In [23]:
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)

In [ ]:
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])

df

In [ ]:
# gca stands for 'get current axis'
ax = plt.gca()

df.plot(kind='line',x='date',y='mean', color='red', ax=ax, figsize=(25,7))
plt.title('Monthly Median Snow Cover for Victor Mine',fontsize=20)
plt.xlabel('date', fontsize=20)
plt.ylabel('monthly mean Snow Cover', fontsize=20)
plt.show()